## Convert PDF to Text
This notebooks shows you how you can extract text from notebook.

### Installing pre-required libs

In [152]:
!pip install PdfReader

In [2]:
### Importing pre-required libs
import PyPDF2

<span style="color:blueviolet">Step 1. By using below method you can get data from pdf</span>

In [4]:
def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        combined_text = ""
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text = page.extract_text()
            combined_text += text

        return combined_text

# Usage example
pdf_path = '1_55661.pdf'



<span style="color:blueviolet">Step 2. By using below method you will get separate paragraphs</span>

In [82]:
def separate_paragraphs(text):
    paragraphs = text.split("  \n")  # Split text by double line breaks

    # Remove leading and trailing whitespaces from each paragraph
    paragraphs = [paragraph.strip() for paragraph in paragraphs]

    return paragraphs


<span style="color:blueviolet">Step 3. Counting Total Paragraphs from pdf</span>

In [137]:
full_doc = extract_text_from_pdf(pdf_path)
full_paras = separate_paragraphs(full_doc)
len(full_paras)

283

In [96]:
question = "what are the supported platform for ssh runner?"

### Retrieve the Context from Text 

In [187]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar_text(sentence, text_objects):
    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer on text_objects
    tfidf_matrix = vectorizer.fit_transform(text_objects)

    # Transform the input sentence using the fitted vectorizer
    sentence_vector = vectorizer.transform([sentence])

    # Calculate cosine similarities between the sentence vector and all text objects
    similarities = cosine_similarity(sentence_vector, tfidf_matrix)

    # Find the index of the most similar text object
    most_similar_index = similarities.argmax()

    # Get the most similar text object
    most_similar_text = text_objects[most_similar_index]

    # Check the length of the most similar text object
    if len(most_similar_text) < 2000:
        # Find the indices and similarities of the related text objects
        sorted_indices = similarities.argsort()[0][::-1]
        sorted_similarities = similarities[0, sorted_indices]

        # Iterate over the related text objects and add paragraphs until length exceeds 2000 characters
        for idx, similarity in zip(sorted_indices, sorted_similarities):
            if idx != most_similar_index:
                related_text = text_objects[idx]
                if len(most_similar_text) + len(related_text) + 2 > 2000:  # +2 for "\n\n"
                    break
                most_similar_text += "\n\n" + related_text

    return most_similar_text
question = "what are the supported platform for ssh runner?"

most_relevant_text = find_most_similar_text(question, full_paras)
print("Most Relevant context:", most_relevant_text)
len(most_relevant_text)

Most Relevant context: First, download the gitlab-runner distribution for the appropriate platform at 
https://docs.gitlab.com/runner/install/ . In this configura tion, the SSH GitLab runner is installed on a Linux 
x86 Red Hat distribution, but similar installation and configuration can be performed on other supported 
platforms.

The SSH runners are executed on a supported platform (Windows, Linux or MacOS) and connect to a 
target machine through SSH for the pipeline execution. In that configuration, SSH runners act like 
gateways to connect platforms: the GitLab server will send the pipeline actions to the SSH runner which 
will forward them to the target machine, in this case th e z/OS environment. The different stages of the 
pipeline will be executed on the target z/OS machine and results will be sent back to the GitLab server 
through the same mechanism.

Provide the GitLab server URL and the registration token to register the runner. Then provide a description 
for this runner

1789